# 1. Authenticate & Import

In [1]:
import os
# 设置代理端口，根据你使用的科学上网软件提供的端口进行
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:10809'
os.environ['HTTPS_PROXY'] = 'https://127.0.0.1:10809'

import ee
# 网不好的时候自动重试ee.Initialize()
print("Try init ee...", end=" ")
eeInited, tryCounts = False, 0
while not eeInited:
    print("%d" % tryCounts, end=" ")
    try:
        ee.Initialize()
    except Exception as e:
        if (str(e)[0:23]=="Please authorize access"):
            ee.Authenticate()
        tryCounts += 1
    else:
        eeInited = True
        print("\nee initialized!")

import geemap

Try init ee... 0 1 2 
ee initialized!


In [2]:
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import cv2

import gdal
import utils
import common
import dataset
import GetMapTiles

# 2. Import Earth Engine Data

In [3]:
# Boundary and Grid-----------------------------------------------------------------------------------------------------
worldBoundary = ee.FeatureCollection("users/liuph/shape/WorldBoundary")
ChinaBoundary = ee.FeatureCollection("users/410093033/China")
WorldGrid5d = ee.FeatureCollection("users/liuph/shape/WorldGrid5dC5")

# Landcover products----------------------------------------------------------------------------------------------------

gong = ee.ImageCollection('users/wangyue/Gong2017Glc30') # gong's data
forest_gong = ee.ImageCollection(gong).qualityMosaic("b1").expression("b(0)==2?1:0").rename("Forestgong")

dataset = ee.ImageCollection('users/sunly3456/Forest2018ImageCollection') # ygs's data
forest_ygs =  ee.ImageCollection(dataset).qualityMosaic("b1").expression("b(0)==1?1:0").rename("Forestygs")

liu = ee.ImageCollection('users/wangyue/Glc2020Fcs30').select('b1') # liu's data
forest_liu = ee.ImageCollection(liu).qualityMosaic("b1").expression("b(0)>=50 && b(0)<=90?1:0").rename("Forestliu")

lc = ee.ImageCollection('users/sunly3456/GLC2020') # Chen's data
forest_chen = ee.ImageCollection(lc).qualityMosaic("b1").expression("b(0)==20?1:0").rename("Forestchen")

dataset = ee.Image('UMD/hansen/global_forest_change_2019_v1_7') # Hansen's data
start2000 = ee.Image(dataset).select('treecover2000').expression("b(0)>30&&b(0)<=100?1:0").rename("start00")
loss00_19 = ee.Image(dataset).expression("b(3)>1&&b(3)<=19?1:0").rename("loss00_19")
gain00_12 = ee.Image(dataset).expression("b(2)==1?1:0").rename("gain00_12")
forest_hs = start2000.add(gain00_12).subtract(loss00_19).expression("b(0)>0?1:0").rename("Foresths")

# Fusion of landcover products------------------------------------------------------------------------------------------
# forest_fuse = forest_gong.add(forest_ygs).add(forest_liu).add(forest_chen).add(forest_hs).rename("ForestFuse")
# forest_fuse = ee.ImageCollection('users/sysushiqian/forestFuse2020').min().rename("forest_fuse")
forest_fuse = ee.ImageCollection('users/sysushiqian/forestFuse2020_new').min().rename("ForestFuse")
forest23 = forest_fuse.expression('b(0)>=2 && b(0)<=3')

# Landsat8, cloud mask, median; add NDVI,NDWI,slop ---------------------------------------------------------------------
Landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
selVar = ee.List(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa'])
LC = Landsat.filter(ee.Filter.calendarRange(2020, 2020, 'year')).select(selVar).map(utils.maskL8sr)
selVar = ee.List(['B2', 'B3', 'B4', 'B5', 'B6', 'B7'])
LC = LC.select(selVar).median()

ndvi = LC.normalizedDifference(['B5', 'B4']).rename('NDVI')
ndwi = LC.normalizedDifference(['B3', 'B5']).rename('NDWI')
DEM = ee.Image("MERIT/DEM/v1_0_3")
terrain = ee.Algorithms.Terrain(DEM)
slope = terrain.select('slope')
stratified_classes = ndvi.expression('(b(0)/0.2)').int().rename('STRATIFIED_CLASSES')

# Composite Image ------------------------------------------------------------------------------------------------------
LC_STN = LC.addBands(ndvi).addBands(ndwi).addBands(slope).addBands(stratified_classes).float()
selVar1 = ee.List(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','slope'])

# Validation points ----------------------------------------------------------------------------------------------------
# valPoints_all = ee.FeatureCollection("users/410093033/wuzhengyi3").merge(ee.FeatureCollection("users/410093033/youzhengyi1"))
# valPoints_all = ee.FeatureCollection("users/410093033/AllMark1Valid")
validpoint1 = ee.FeatureCollection('users/410093033/AllMarkValid').filterMetadata('Type','less_than',5)
validpoint2 = ee.FeatureCollection('users/sunly3456/globalValidPoints/certainSample0121').filterMetadata('Type','less_than',5)
valPoints_all = validpoint1.merge(validpoint2)

In [4]:
ID5d_list = [355,356,361,403,404,432,433,434,436,457,459,464,466,473,490,494,495,496,497,498,526,529,532,538,564,573,595,625,681,722,735,736,739,773,774,780,816,826,827,850,870,886,895,908,940,1019,1021,1022,1023,1114,1127,1133,1140,1181,1182,1187,1195,1197]

In [5]:
for i in ID5d_list: 
    ID5d = i
    dataPath = r"I:/ff0105_ge01"

    time_start = time.time()
    dataPath_curGrid = dataPath + "/grid%d" % ID5d
    if not os.path.exists(dataPath_curGrid):
        os.mkdir(dataPath_curGrid)
    selected5d = WorldGrid5d.filterMetadata("ID", "equals", ID5d).first()
    llLng, llLat = selected5d.getNumber("llLng").getInfo(), selected5d.getNumber("llLat").getInfo()
    grid05d, grid01d = utils.get0501Grid(selected5d, forest23)

    IDlist_01d = grid01d.reduceColumns(ee.Reducer.toList(), ["ID_01d"]).get("list").getInfo()

    for tileNum in range(10):
        # 1. 生成随机森林样本-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
        idList_05d = list(range(tileNum*10, (tileNum+1)*10, 1))
        idList_01d = IDlist_01d[tileNum*10:(tileNum+1)*10]
        print("\n[Grid%d:Tile%d] %s" % (ID5d, tileNum, time.ctime()))
        print("%s" % "-"*80)

        # if not all needed ge/ff01/ff05 are found, wait until all needs are satisfied
        while(not common.checkExists_ffge(dataPath, ID5d, idList_05d, idList_01d)):
            print("\ndata missed, download started...")
            # time.sleep(120)
            for id05d in idList_05d:
                f05d = grid05d.filterMetadata("ID_05d", "equals", id05d).first()
                f01d = grid01d.filterMetadata("ID_05d", "equals", id05d).first()
                common.download_ffge(dataPath, feature05=f05d, feature01=f01d, forest_fuse=forest_fuse)
            
        print("\ndata check paased!")
        


[Grid355:Tile0] Sat Mar 20 21:54:23 2021
--------------------------------------------------------------------------------
missing data: GE FF01 FF05, waiting for download...

data missed, download started...
Total number：39 X 47 DownLoading... 0% 